In [14]:
import tensorflow as tf
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/RNN_variableLength_Sequence')
import pickle
from prepare_data import parse_seq
# from prepare_data2 import parse_seq

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !python3 prepare_data2.py [-m 100] shakespeare_input.txt sp \\n

In [16]:
!python3 prepare_data.py shakespeare_input.txt sk

2020-06-01 20:58:12.631217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [17]:
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("sk.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))
# data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("sk_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'C': 1, '!': 2, '.': 3, ' ': 4, 'G': 5, 'f': 6, 'i': 7, 'X': 8, 'I': 9, 'N': 10, 'c': 11, ']': 12, 'Z': 13, '[': 14, 'B': 15, 'L': 16, 'g': 17, 'K': 18, ':': 19, '&': 20, ';': 21, 'd': 22, 'O': 23, ',': 24, 'Y': 25, 'Q': 26, 'z': 27, 'h': 28, 'q': 29, 'l': 30, "'": 31, 's': 32, 'A': 33, 'V': 34, '-': 35, 'n': 36, 'v': 37, 'u': 38, 'r': 39, 'j': 40, 't': 41, 'a': 42, 'y': 43, 'M': 44, 'W': 45, 'H': 46, 'T': 47, 'e': 48, 'p': 49, '3': 50, 'k': 51, 'S': 52, '?': 53, 'E': 54, 'R': 55, '\n': 56, 'b': 57, 'U': 58, 'x': 59, 'o': 60, 'w': 61, 'D': 62, 'P': 63, 'J': 64, '$': 65, 'm': 66, 'F': 67, '<S>': 0}
68


In [18]:
for ind, thing in enumerate(data):
    inds = thing.numpy()
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    # print(ind)
    print(to_chars)
    #print()
    break

<S>First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, yo


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = data.map(split_input_target)
dataset

In [20]:
# for input_example, target_example in  dataset.take(1):
#   print ('Input data: ', repr(''.join(idx2chr[input_example.numpy()])))
#   print ('Target data:', repr(''.join(ind_to_ch[target_example.numpy()])))

# to_chars = "".join([ind_to_ch[ind] for ind in inds])

for input_example, target_example in  dataset.take(1):
  input_example = input_example.numpy().flatten()
  target_example = target_example.numpy().flatten()
  print ('Input data :',  repr("".join([ind_to_ch[ind] for ind in input_example])))
  print ('Target data:', repr("".join([ind_to_ch[ind] for ind in target_example])))

Input data : '<S>First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, y'
Target data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, yo'


In [21]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(ind_to_ch[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(ind_to_ch[target_idx])))

Step    0
  input: 0 ('<S>')
  expected output: 67 ('F')
Step    1
  input: 67 ('F')
  expected output: 7 ('i')
Step    2
  input: 7 ('i')
  expected output: 39 ('r')
Step    3
  input: 39 ('r')
  expected output: 32 ('s')
Step    4
  input: 32 ('s')
  expected output: 41 ('t')


In [ ]:
dataset = dataset.shuffle(22981).batch(128, drop_remainder= True)
embedding_dim = 512
rnn_units = 1024
batch_size = 128

In [ ]:
# dataset = dataset.shuffle(4657).padded_batch(128, padded_shapes= 200, drop_remainder= True)
# embedding_dim = 512
# rnn_units = 1024
# batch_size = 128

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential(name= 'GRU')
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.GRU(rnn_units, return_sequences= True, stateful= True, recurrent_initializer= 'glorot_uniform'))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          34816     
_________________________________________________________________
gru (GRU)                    (128, None, 1024)         4724736   
_________________________________________________________________
dense (Dense)                (128, None, 68)           69700     
Total params: 4,829,252
Trainable params: 4,829,252
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_ind = tf.random.categorical(example_batch_predictions[0], num_samples= 1)
sampled_ind = tf.squeeze(sampled_ind, axis= -1).numpy()

# sampled_ind
input_example_batch1 = input_example_batch[0].numpy()
# input_example_batch1
print("Input: \n", repr("".join([ind_to_ch[ind] for ind in input_example_batch1])))
print()
print("Next Char Predictions: \n", repr("".join([ind_to_ch[ind] for ind in sampled_ind])))

(128, 199, 68) # (batch_size, sequence_length, vocab_size)
Input: 
 '<S>e the ruin of your love\nAnd be my vantage to exclaim on you.\n\nBASSANIO:\nMadam, you have bereft me of all words,\nOnly my blood speaks to you in my veins;\nAnd there is such confusion in my powers,\nAs '

Next Char Predictions: 
 "iTJW?EabZPVQaUwqas\nb3LbGEQY-ffPZ.!\n\n:B'CNIK$Tpi?LJx.NSUeYCYBzU&wYKC\nnv?uloD?;lTtvUpQmDR33LbgkaKtR'HLhJd$Q&c\nv;Btj?'lVKaWF?F]$'znzTp3gjBcHu?JPccjXNdS!TOScyvl;;DhvpdnZ<S> :g'JGKKRJ\nwD'MG-A.n,xXW3FWsidx?K"


In [26]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits= True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Mean Loss: ", example_batch_loss.numpy().mean())

Mean Loss:  4.220621


In [27]:
model.compile(optimizer= 'Adam', loss= loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs= 10, callbacks=[checkpoint_callback])

Epoch 1/10
179/179 [==============================] - 90s 501ms/step - loss: 2.5429
Epoch 2/10
179/179 [==============================] - 90s 500ms/step - loss: 1.8072
Epoch 3/10
179/179 [==============================] - 90s 502ms/step - loss: 1.5486
Epoch 4/10
179/179 [==============================] - 90s 500ms/step - loss: 1.4256
Epoch 5/10
179/179 [==============================] - 90s 502ms/step - loss: 1.3580
Epoch 6/10
179/179 [==============================] - 90s 502ms/step - loss: 1.3147
Epoch 7/10
179/179 [==============================] - 90s 501ms/step - loss: 1.2827
Epoch 8/10
179/179 [==============================] - 90s 503ms/step - loss: 1.2570
Epoch 9/10
179/179 [==============================] - 90s 500ms/step - loss: 1.2348
Epoch 10/10
179/179 [==============================] - 90s 501ms/step - loss: 1.2156


In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size= 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            34816     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
dense_1 (Dense)              (1, None, 68)             69700     
Total params: 4,829,252
Trainable params: 4,829,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      # text_generated.append(idx2char[predicted_id])
      text_generated.append(ind_to_ch[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
print(generate_text(model, "JULIET: "))

JULIET: 

HELENA:
Of nothing, it's good, but on their neighbour,
And survy your thoughts to more names:--but, to cry so
That I at still never leave them as I
Hang it me. Good Master Sly! o heavens!
His creuts makes lords and unspossily.

APEMANTUS:
Pray, I'll be glad of thine all: where's aught?

WESTMIRET:
You shall not, to receive
Your friend wild conjoiced by this comfort,
Whose wing, when his cruelity and hervelles
like refuse, till he fall of heaving as married men's
open, wickedny of the cold fortunes that of
such a kingdom? stains so weak a maid most true, take
I as many fearfully unto this sport,
And I revoice good month, I'll do thy fit:
I am not it against some of Cordal,
Thou shalt enluce his troop. Neither! Flattering sleep,
As to 'twere: and therefore anster such service.

MISTRESS PAGE:
Has promised you as he rich: if you after day.

CLEON:

SPEED:
Why, follow me, and thou ways us, for she cried,
And mine have none farewell.

Fipst
Lords, then, if you know thou liegest ma